In [23]:
import csv
import numpy as np
import json
import os

In [33]:
!python -m src.collect --experiment-name "jz/greedy/rule" --dataset "pain"

In [38]:
!python -m src.collect --experiment-name "jz/greedy/no_rule" --dataset "pleasure"
!python -m src.collect --experiment-name "jz/greedy/no_rule" --dataset "deceive"
!python -m src.collect --experiment-name "jz/greedy/no_rule" --dataset "cheat"
!python -m src.collect --experiment-name "jz/greedy/no_rule" --dataset "break_promise"
!python -m src.collect --experiment-name "jz/greedy/no_rule" --dataset "break_law"
!python -m src.collect --experiment-name "jz/greedy/no_rule" --dataset "duty"
!python -m src.collect --experiment-name "jz/greedy/no_rule" --dataset "kill"
!python -m src.collect --experiment-name "jz/greedy/no_rule" --dataset "pain"
!python -m src.collect --experiment-name "jz/greedy/no_rule" --dataset "disable"
!python -m src.collect --experiment-name "jz/greedy/no_rule" --dataset "freedom"

In [24]:
base_path = "data/responses/jz/"
models = ["google_flan-t5-small", "google_flan-t5-base", "google_flan-t5-large","google_flan-t5-xl", "google_flan-t5-xxl","bigscience_bloomz-560m", "bigscience_bloomz-1b1", "bigscience_bloomz-1b7", "bigscience_bloomz-3b", "bigscience_bloomz-7b1"]
models_flan = ["google_flan-t5-small", "google_flan-t5-base", "google_flan-t5-large"]
models_temp = ["google_flan-t5-small", "google_flan-t5-base", "google_flan-t5-large", "bigscience_bloomz-560m", "bigscience_bloomz-1b1", "bigscience_bloomz-1b7"]
datasets = ["dplow", "privacylow", "dignitylow", "secretlow"]
questions_forms, questions_forms_d, questions_forms_p  = ["ab", "repeat", "compare"], ["ab_d", "repeat_d", "compare_d"], ["ab_p", "repeat_p", "compare_p"]
privacy_ids = [f"PP_00{i}" for i in range(1, 10)] + [f"PP_0{i}" for i in range(10, 31)]
dignity_ids = [f"D_00{i}" for i in range(1, 10)] + [f"D_0{i}" for i in range(10, 37)]
secret_ids = [f"PS_00{i}" for i in range(1, 10)] + [f"PS_0{i}" for i in range(10, 31)]
laws = ["kill", "pain", "disable", "freedom", "pleasure", "deceive", "cheat", "break_promise", "break_law", "duty"]
laws_dp = laws + ["dignity", "privacy"]
number_of_situations = {"kill" : 53, "pain" : 307, "disable" : 70, "freedom" : 96, "pleasure" : 166, "deceive" : 244, "cheat" : 74, "break_promise" : 62, "break_law" : 150, "duty" : 435, "dignity" : 36, "privacy" : 30, "rule_dignity" : 36, "rule_privacy" : 30}



In [25]:
privacy_ids = [f"PP_00{i}" for i in range(1, 10)] + [f"PP_0{i}" for i in range(10, 31)]
dignity_ids = [f"D_00{i}" for i in range(1, 10)] + [f"D_0{i}" for i in range(10, 37)]

In [26]:
def gather_scenario_ids(law : str, folder : str = "paperlaws") -> list[str]:
    path = f"data\{folder}_scenarios\moralchoice_{law}_ambiguity.csv"
    results = []
    with open(path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            results.append(row[0])
    return results

def action_likelihood(action : int, model : str, law : str, rule : bool, method : str) -> np.array:
    # action = 0 for action1, 1 for action2
    if law == "dignity" :
        ids = dignity_ids
    elif law == "privacy" :
        ids = privacy_ids
    else :
        ids = gather_scenario_ids(law)
    likelihoods = []
    if rule : x = "rule"
    else : x = "no_rule"
    try :
        path = f"data/responses/jz\{method}\{x}\{law}\{model}.csv"
    except FileNotFoundError:
        print(f"File not found: {law}/{model}.csv")
        return -1
    
    for scenario_id in ids:
        actions = [0,0]
        with open(path, 'r') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                if row[1] == scenario_id:
                    if row[3] in ['ab_d','compare_d','repeat_d','ab_p','compare_p','repeat_p'] :
                        if 'action1' in row : actions[0] += 1
                        elif 'action2' in row : actions[1] += 1
                    elif row[3] in ['ab','compare','repeat'] :
                        if 'action1' in row : actions[0] += 1
                        elif 'action2' in row : actions[1] += 1
        if actions[0] == 0 and actions[1] == 0 : print(scenario_id)
        sum = actions[0] + actions[1]
        try :
            actions[0] /= sum
            actions[1] /= sum
            likelihoods.append(actions[action])
        except ZeroDivisionError:
            print(scenario_id)
    return likelihoods

In [44]:
import os
import json
import numpy as np

def store_to_json(action: int, model: str, law: str, rule: bool, method: str) -> None:
    likelihoods = action_likelihood(action, model, law, rule, method)
    
    # Convert NumPy array to a list for JSON serialization
    if isinstance(likelihoods, np.ndarray):
        likelihoods = likelihoods.tolist()
    
    x = "rule" if rule else "no_rule"
    
    # Define the directory and file path
    directory = f"jz/{method}/{x}"
    os.makedirs(directory, exist_ok=True)
    file_path = f"{directory}/results.json"
    
    # Load existing data if the file already exists
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            data = json.load(file)
    else:
        data = {}
    
    # Update the data with the new entry
    if law not in data:
        data[law] = {}
    
    data[law][model] = likelihoods
    
    # Save the updated data back to the JSON file
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

# Example usage (ensure action_likelihood is defined elsewhere)
# store_to_json(5, 'model1', 'law1', True, 'methodA')

# store_to_json(0, "google_flan-t5-xl", "cheat", True, "top_p")

# #create a list with laws without kill and pain
# list = []
# # for law in ["disable", "duty", "kill", "pain"]:
# for law in ["disable", "duty", "kill", "pain"]:
#     for model in models:
#             for method in ["greedy"]:
#                 if not(check_all(0, model, law, False, method)) :
#                     print(f"Missing {law} {model} {method} rule")

In [55]:
#make a law list without pain, duty, deceive
laws2 = ["kill","disable", "freedom", "pleasure", "cheat", "break_promise", "break_law"]
trois = ["pain","deceive", "duty"]
for law in laws:
    print(law)
    for model in models:
        for method in ["top_p"]:
            try : store_to_json(0, model, law, True, method)
            except FileNotFoundError: print(f"Missing {law} {model} {method} rule")

kill
C_101
C_101
C_107
C_107
C_109
C_109
C_003
C_003
C_010
C_010
C_014
C_014
C_015
C_015
C_019
C_019
C_020
C_020
C_021
C_021
C_027
C_027
C_028
C_028
C_030
C_030
C_031
C_031
C_035
C_035
C_041
C_041
C_043
C_043
C_045
C_045
C_046
C_046
C_050
C_050
C_051
C_051
C_052
C_052
C_055
C_055
C_056
C_056
C_058
C_058
C_059
C_059
C_063
C_063
C_075
C_075
C_077
C_077
C_081
C_081
C_096
C_096
C_101
C_101
C_109
C_109
C_110
C_110
C_123
C_123
C_279
C_279
C_314
C_314
C_336
C_336
C_1117
C_1117
C_1153
C_1153
C_1170
C_1170
C_1190
C_1190
C_1207
C_1207
C_1231
C_1231
pain
Missing pain google_flan-t5-small top_p rule
Missing pain google_flan-t5-base top_p rule
Missing pain google_flan-t5-large top_p rule
Missing pain google_flan-t5-xl top_p rule
Missing pain google_flan-t5-xxl top_p rule
Missing pain bigscience_bloomz-560m top_p rule
Missing pain bigscience_bloomz-1b1 top_p rule
Missing pain bigscience_bloomz-1b7 top_p rule
Missing pain bigscience_bloomz-3b top_p rule
Missing pain bigscience_bloomz-7b1 top_p rule
d

KeyboardInterrupt: 

In [42]:
def check_all(action : int, model : str, law : str, rule : bool, method : str) -> bool:
    if law == "dignity" :
        ids = dignity_ids
    elif law == "privacy" :
        ids = privacy_ids   
    else :
        ids = gather_scenario_ids(law) 
    if rule : x = "rule"
    else : x = "no_rule"
    path = f"data/responses/jz\{method}\{x}\{law}\{model}.csv"
    if not os.path.exists(path):
        #print(f"File not found: {law}/{model}.csv")
        return False
    list = []
    with open(path, 'r') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                if row[1] in ids and row[1] not in list:
                    list.append(row[1])
    print(f"{model}, {len(list)}")
    if len(list) == len(ids) : return True
    else : return False

for law in ["pain"]:
    print(f"\n Checking {law}")
    for method in ["greedy"]:
        for model in models:
                    if not(check_all(0, model, law, False, method)) :
                        print(f"Missing {law} {model} {method} no rule")
                    if not(check_all(0, model, law, True, method)) :
                        print(f"Missing {law} {model} {method} rule")


 Checking pain
google_flan-t5-small, 307
google_flan-t5-small, 307
google_flan-t5-base, 307
google_flan-t5-base, 307
google_flan-t5-large, 307
google_flan-t5-large, 307
Missing pain google_flan-t5-xl greedy no rule
google_flan-t5-xl, 307
google_flan-t5-xxl, 127
Missing pain google_flan-t5-xxl greedy no rule
google_flan-t5-xxl, 285
Missing pain google_flan-t5-xxl greedy rule
bigscience_bloomz-560m, 307
bigscience_bloomz-560m, 307
bigscience_bloomz-1b1, 307
bigscience_bloomz-1b1, 307
bigscience_bloomz-1b7, 307
bigscience_bloomz-1b7, 307
bigscience_bloomz-3b, 307
bigscience_bloomz-3b, 307
bigscience_bloomz-7b1, 307
bigscience_bloomz-7b1, 307


In [31]:
!python -m src.collect --experiment-name "jz/greedy/no_rule" --dataset "deceive"

In [29]:
def check_all(action : int, model : str, law : str, rule : bool, method : str) -> bool:
    if law == "dignity" :
        ids = dignity_ids
    elif law == "privacy" :
        ids = privacy_ids   
    else :
        ids = gather_scenario_ids(law) 
    if rule : x = "rule"
    else : x = "no_rule"
    path = f"data/responses/jz\{method}\{x}\{law}\{model}.csv"
    if not os.path.exists(path):
        #print(f"File not found: {law}/{model}.csv")
        return False
    list = []
    with open(path, 'r') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                if row[1] in ids and row[1] not in list:
                    list.append(row[1])
    if len(list) == len(ids) : return True
    else : return False

for law in laws:
    print(f"\n Checking {law}")
    for method in ["greedy"]:
        for model in models:
                    if (check_all(0, model, law, False, method)) :
                        print(f"YEEEEES {law} {model} {method} no rule")
                    if not(check_all(0, model, law, True, method)) :
                        print(f"Missing {law} {model} {method} rule")


 Checking kill
YEEEEES kill google_flan-t5-small greedy no rule
YEEEEES kill google_flan-t5-base greedy no rule
YEEEEES kill google_flan-t5-large greedy no rule
YEEEEES kill google_flan-t5-xl greedy no rule
YEEEEES kill google_flan-t5-xxl greedy no rule
Missing kill google_flan-t5-xxl greedy rule
YEEEEES kill bigscience_bloomz-560m greedy no rule
YEEEEES kill bigscience_bloomz-1b1 greedy no rule
YEEEEES kill bigscience_bloomz-1b7 greedy no rule
YEEEEES kill bigscience_bloomz-3b greedy no rule
YEEEEES kill bigscience_bloomz-7b1 greedy no rule

 Checking pain
YEEEEES pain google_flan-t5-small greedy no rule
YEEEEES pain google_flan-t5-base greedy no rule
YEEEEES pain google_flan-t5-large greedy no rule
Missing pain google_flan-t5-xxl greedy rule
YEEEEES pain bigscience_bloomz-560m greedy no rule
YEEEEES pain bigscience_bloomz-1b1 greedy no rule
YEEEEES pain bigscience_bloomz-1b7 greedy no rule
YEEEEES pain bigscience_bloomz-3b greedy no rule
YEEEEES pain bigscience_bloomz-7b1 greedy no 